In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.feature_selection import SequentialFeatureSelector

In [2]:
df = pd.read_excel('ToyotaCorolla2.xls',sheet_name= 'data')

# movies_0 = pd.read_excel('ToyotaCorolla2.xls',sheet_name=data)

In [3]:
df.head()

,Id,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,1,13500,23,10,2002,46986,90,1,0,2000,...,1,1,1,0,0,0,1,0,0,0
1,2,13750,23,10,2002,72937,90,1,0,2000,...,1,0,1,0,0,0,1,0,0,0
2,3,13950,24,9,2002,41711,90,1,0,2000,...,0,0,1,0,0,0,1,0,0,0
3,4,14950,26,7,2002,48000,90,0,0,2000,...,0,0,1,0,0,0,1,0,0,0
4,5,13750,30,3,2002,38500,90,0,0,2000,...,1,1,1,0,1,0,1,0,0,0


In [4]:
header_texts = pd.read_excel('ToyotaCorolla2.xls',sheet_name= 'variables')

In [5]:
header_texts.head(40)

,Variable,Description
0,Id,Record_ID
1,Model,Model Description
2,Price,Offer Price in EUROs
3,Age_08_04,Age in months as in August 2004
4,Mfg_Month,Manufacturing month (1-12)
5,Mfg_Year,Manufacturing Year
6,KM,Accumulated Kilometers on odometer
7,Fuel_Type,"Fuel Type (Petrol, Diesel, CNG)"
8,HP,Horse Power
9,Met_Color,"Metallic Color? (Yes=1, No=0)"


In [6]:
df.shape


(1436, 35)

In [7]:
df.columns

Index(['Id', 'Price', 'Age_08_04', 'Mfg_Month', 'Mfg_Year', 'KM', 'HP',
       'Met_Color', 'Automatic', 'cc', 'Doors', 'Cylinders', 'Gears',
       'Quarterly_Tax', 'Weight', 'Mfr_Guarantee', 'BOVAG_Guarantee',
       'Guarantee_Period', 'ABS', 'Airbag_1', 'Airbag_2', 'Airco',
       'Automatic_airco', 'Boardcomputer', 'CD_Player', 'Central_Lock',
       'Powered_Windows', 'Power_Steering', 'Radio', 'Mistlamps',
       'Sport_Model', 'Backseat_Divider', 'Metallic_Rim', 'Radio_cassette',
       'Tow_Bar'],
      dtype='object')

In [8]:
df.describe()

,Id,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
count,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.00000,...,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000
mean,721.555014,10730.824513,55.947075,5.548747,1999.625348,68533.259749,101.502089,0.674791,0.055710,1576.85585,...,0.580084,0.561978,0.977716,0.146240,0.256964,0.300139,0.770195,0.204735,0.145543,0.277855
std,416.476890,3626.964585,18.599988,3.354085,1.540722,37506.448872,14.981080,0.468616,0.229441,424.38677,...,0.493717,0.496317,0.147657,0.353469,0.437111,0.458478,0.420854,0.403649,0.352770,0.448098
min,1.000000,4350.000000,1.000000,1.000000,1998.000000,1.000000,69.000000,0.000000,0.000000,1300.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,361.750000,8450.000000,44.000000,3.000000,1998.000000,43000.000000,90.000000,0.000000,0.000000,1400.00000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,721.500000,9900.000000,61.000000,5.000000,1999.000000,63389.500000,110.000000,1.000000,0.000000,1600.00000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,1081.250000,11950.000000,70.000000,8.000000,2001.000000,87020.750000,110.000000,1.000000,0.000000,1600.00000,...,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,1442.000000,32500.000000,80.000000,12.000000,2004.000000,243000.000000,192.000000,1.000000,1.000000,16000.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
df.dtypes

Id                  int64
Price               int64
Age_08_04           int64
Mfg_Month           int64
Mfg_Year            int64
KM                  int64
HP                  int64
Met_Color           int64
Automatic           int64
cc                  int64
Doors               int64
Cylinders           int64
Gears               int64
Quarterly_Tax       int64
Weight              int64
Mfr_Guarantee       int64
BOVAG_Guarantee     int64
Guarantee_Period    int64
ABS                 int64
Airbag_1            int64
Airbag_2            int64
Airco               int64
Automatic_airco     int64
Boardcomputer       int64
CD_Player           int64
Central_Lock        int64
Powered_Windows     int64
Power_Steering      int64
Radio               int64
Mistlamps           int64
Sport_Model         int64
Backseat_Divider    int64
Metallic_Rim        int64
Radio_cassette      int64
Tow_Bar             int64
dtype: object

In [10]:
# feature_cols = [x for x in df.columns if (x != 'Id') and (x!= 'Price')]

feature_cols = [x for x in df.columns if x!= 'Price']
X = df[feature_cols]
y = df['Price']


In [11]:
X.head()

,Id,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,1,23,10,2002,46986,90,1,0,2000,3,...,1,1,1,0,0,0,1,0,0,0
1,2,23,10,2002,72937,90,1,0,2000,3,...,1,0,1,0,0,0,1,0,0,0
2,3,24,9,2002,41711,90,1,0,2000,3,...,0,0,1,0,0,0,1,0,0,0
3,4,26,7,2002,48000,90,0,0,2000,3,...,0,0,1,0,0,0,1,0,0,0
4,5,30,3,2002,38500,90,0,0,2000,3,...,1,1,1,0,1,0,1,0,0,0


In [12]:
a=df.copy()
a.head()

,Id,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,1,13500,23,10,2002,46986,90,1,0,2000,...,1,1,1,0,0,0,1,0,0,0
1,2,13750,23,10,2002,72937,90,1,0,2000,...,1,0,1,0,0,0,1,0,0,0
2,3,13950,24,9,2002,41711,90,1,0,2000,...,0,0,1,0,0,0,1,0,0,0
3,4,14950,26,7,2002,48000,90,0,0,2000,...,0,0,1,0,0,0,1,0,0,0
4,5,13750,30,3,2002,38500,90,0,0,2000,...,1,1,1,0,1,0,1,0,0,0


In [13]:
a.drop('Price',axis=1,inplace=True)

In [14]:
a.head()

,Id,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,1,23,10,2002,46986,90,1,0,2000,3,...,1,1,1,0,0,0,1,0,0,0
1,2,23,10,2002,72937,90,1,0,2000,3,...,1,0,1,0,0,0,1,0,0,0
2,3,24,9,2002,41711,90,1,0,2000,3,...,0,0,1,0,0,0,1,0,0,0
3,4,26,7,2002,48000,90,0,0,2000,3,...,0,0,1,0,0,0,1,0,0,0
4,5,30,3,2002,38500,90,0,0,2000,3,...,1,1,1,0,1,0,1,0,0,0


In [15]:
p= len(X.columns)
print(p)
n= len(X.index)
print(n)

34
1436


In [16]:
# for X Fulldata

reg = LinearRegression().fit(X, y)
print(reg.score(X, y))
print(reg.coef_)
print(reg.intercept_)

y_pred = reg.predict(X)

print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
print("R2: %.2f" % r2_score(y, y_pred))

R2_t_fulldata= reg.score(X, y)
coefs_fulldata = reg.coef_
intercep_fulldata = reg.intercept_

y_pred = reg.predict(X)
mse = mean_squared_error(y, y_pred)
r2= r2_score(y, y_pred)
Adj_r2 = 1-(1-r2)*((n-1)/(n-p-1))

result_fulldata = pd.DataFrame( columns = ['case', 'R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

case_fulldata = 'All features'

result_fulldata=result_fulldata.append({'case': case_fulldata, 'R2 score': R2_t_fulldata, 'intercepts': intercep_fulldata, 
                'coffeients': coefs_fulldata,'Mean squared error': mse ,'R2' : r2, 'Adj_r2': Adj_r2}, ignore_index=True)




result_fulldata.head()


0.9083547203791587
[ 1.60601057e+00  9.13261204e+08  9.13261228e+08  1.09591363e+10
 -1.34332141e-02  2.53112041e+01 -3.87074132e+01  4.07633577e+02
 -1.00604016e-01  6.96390382e+01 -3.20662951e+04  3.62556058e+02
  7.82122225e+00  8.78182819e+00  2.77611490e+02  4.97759874e+02
  5.83347138e+01 -1.73983179e+02  2.50118829e+02 -3.80326269e+01
  1.88873169e+02  2.32808049e+03 -2.50134931e+02  1.90463615e+02
 -1.33298404e+02  4.59163620e+02 -3.24023371e+02  6.89910985e+02
  9.65204060e+00  2.32379868e+02 -8.37880235e+01  1.26535481e+02
 -7.20888268e+02 -1.80228390e+02]
-21970328431613.258
Mean squared error: 1204742.39
R2: 0.91


,case,R2 score,intercepts,coffeients,Mean squared error,R2,Adj_r2
0,All features,0.908355,-2.197033e+13,"[1.6060105730085568, 913261203.5167035, 913261...",1.204742e+06,0.908355,0.906131


In [17]:
# i=0

# result = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','Coefficient of determination'])

# while i < 5:

#     # print('Simulation number :', i)


#     # repeat this activity multiple times and record the R2 and MSE for each run
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

#     reg2 = LinearRegression().fit(X_train, y_train)
#     r2_t= reg2.score(X, y)
#     coefs = reg2.coef_
#     intercep = reg2.intercept_
#     # print(reg2.score(X, y))
#     # print(reg2.coef_)
#     # print(reg2.intercept_)
#     y_pred_test = reg2.predict(X_test)
#     mse = mean_squared_error(y_test, y_pred_test)
#     r2 = r2_score(y_test, y_pred_test)
#     # print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_test))
#     # print("Coefficient of determination: %.3f" % r2_score(y_test, y_pred_test))
#     i= i+1

#     result=result.append({'R2 score': r2_t, 'intercepts': intercep, 'coffeients': coefs,'Mean squared error': mse ,'Coefficient of determination' : r2}, ignore_index=True)





# r2_avg = result['Coefficient of determination'].mean()
# mse_avg = result['Mean squared error'].mean()
# case= 'All features'

# result_avg = pd.DataFrame( columns = ['Case','Average Mean squared error', 'Average Coefficient of determination'])

# result_avg= result_avg.append({'Case': case, 'Average Mean squared error': mse_avg, 'Average Coefficient of determination': r2_avg}, ignore_index=True)

# result.head(12)

# # print(' The total Mean of results after', i , ' number of simulations is :')

# # result_avg.head()
#     # df_f=df_f.append({'Name':name_4s,'Contact':c,'address':d,'url':u,'ratings':rat,'comments':com,'image':image}, ignore_index=True)

In [18]:
# print(' The total Mean of results after', i , ' number of simulations is :')

# result_avg.head()

In [19]:
# result['Coefficient of determination'].mean()

In [20]:
p= len(X.columns)
print(p)
n= len(X.index)
print(n)

34
1436


In [21]:
i=0

result = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'MSE_Train','R2_Train', 'Adj_r2_train' ,
                                'MSE_Test','R2_Test', 'Adj_r2_test' ])

while i < 5:

    # print('Simulation number :', i)


    # repeat this activity multiple times and record the R2 and MSE for each run
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

    reg2 = LinearRegression().fit(X_train, y_train)
    r2_t= reg2.score(X, y)
    coefs = reg2.coef_
    intercep = reg2.intercept_
    # print(reg2.score(X, y))
    # print(reg2.coef_)
    # print(reg2.intercept_)

    y_pred_train = reg2.predict(X_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))






    y_pred_test = reg2.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))





    i= i+1

    result=result.append({'R2 score': r2_t, 'intercepts': intercep, 'coffeients': coefs, 'MSE_Train': mse_train,
                             'R2_Train': r2_train, 'Adj_r2_train': Adj_r2_train, 
                             'MSE_Test': mse_test, 'R2_Test': r2_test, 
                             'Adj_r2_test': Adj_r2_test }, ignore_index=True)





r2_train_avg = result['R2_Train'].mean()
Adj_r2_train_avg= result['Adj_r2_train'].mean()
mse_train_avg = result['MSE_Train'].mean()



r2_test_avg = result['R2_Test'].mean()
Adj_r2_test_avg= result['Adj_r2_test'].mean()
mse_test_avg = result['MSE_Test'].mean()

case= 'All features'

result_avg = pd.DataFrame( columns = ['Case','Average MSE_Train', 'Average R2_Train',
                                        'Adj_r2_train_avg', 'Average MSE_Test', 'Average R2_Test',
                                         'Adj_r2_test_avg'])

result_avg= result_avg.append({'Case': case, 'Average MSE_Train': mse_train_avg, 'Average R2_Train': r2_train_avg,
                                'Adj_r2_train_avg': Adj_r2_train_avg, 'Average MSE_Test': mse_test_avg, 
                                'Average R2_Test': r2_test_avg, 'Adj_r2_test_avg':Adj_r2_test_avg}, ignore_index=True)

result.head(12)

# print(' The total Mean of results after', i , ' number of simulations is :')

# result_avg.head()
    # df_f=df_f.append({'Name':name_4s,'Contact':c,'address':d,'url':u,'ratings':rat,'comments':com,'image':image}, ignore_index=True)

,R2 score,intercepts,coffeients,MSE_Train,R2_Train,Adj_r2_train,MSE_Test,R2_Test,Adj_r2_test
0,0.906857,-4.349135e+04,"[1.7654515696425734, -161.161971184788, -141.9...",1.218132e+06,0.908712,0.906497,1.239121e+06,0.902272,0.899900
1,0.907352,-3.568786e+12,"[1.7937379093473174, 148346916.16670063, 14834...",1.304298e+06,0.907048,0.904792,1.016506e+06,0.908226,0.905998
2,0.907234,-4.163850e+04,"[1.348733753420691, -148.9910621648089, -126.3...",1.212736e+06,0.914751,0.912682,1.235190e+06,0.883501,0.880673
3,0.907305,-4.480103e+04,"[1.8524715469098882, -161.7343839355427, -136....",1.203466e+06,0.911168,0.909012,1.253697e+06,0.896673,0.894165
4,0.792227,-4.792960e+04,"[1.2544663279547748, -141.28597072825818, -120...",1.070773e+06,0.917622,0.915622,6.603371e+06,0.510348,0.498465


In [22]:
print(' The total Mean of results after', i , ' number of simulations is :')

result_avg.head()

 The total Mean of results after 5  number of simulations is :


,Case,Average MSE_Train,Average R2_Train,Adj_r2_train_avg,Average MSE_Test,Average R2_Test,Adj_r2_test_avg
0,All features,1.201881e+06,0.91186,0.909721,2.269577e+06,0.820204,0.81584


In [23]:
X = sm.add_constant(X)
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())



                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.908
Model:                            OLS   Adj. R-squared:                  0.906
Method:                 Least Squares   F-statistic:                     434.6
Date:                Sun, 30 Jan 2022   Prob (F-statistic):               0.00
Time:                        19:41:37   Log-Likelihood:                -12091.
No. Observations:                1436   AIC:                         2.425e+04
Df Residuals:                    1403   BIC:                         2.442e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Id                   1.6050      0.257  

In [24]:
X.columns

Index(['Id', 'Age_08_04', 'Mfg_Month', 'Mfg_Year', 'KM', 'HP', 'Met_Color',
       'Automatic', 'cc', 'Doors', 'Cylinders', 'Gears', 'Quarterly_Tax',
       'Weight', 'Mfr_Guarantee', 'BOVAG_Guarantee', 'Guarantee_Period', 'ABS',
       'Airbag_1', 'Airbag_2', 'Airco', 'Automatic_airco', 'Boardcomputer',
       'CD_Player', 'Central_Lock', 'Powered_Windows', 'Power_Steering',
       'Radio', 'Mistlamps', 'Sport_Model', 'Backseat_Divider', 'Metallic_Rim',
       'Radio_cassette', 'Tow_Bar'],
      dtype='object')

In [25]:
ar=pd.DataFrame(columns = ['Features', 'P_Values'])
c=[]
for l in X.columns:
    b = res.pvalues[l]
    if b > 0.05:

        # result_avg= result_avg.append({'Case': case, 'Average MSE_Train': mse_train_avg, 'Average R2_Train': r2_train_avg,
        #                         'Adj_r2_train_avg': Adj_r2_train_avg, 'Average MSE_Test': mse_test_avg, 
        #                         'Average R2_Test': r2_test_avg, 'Adj_r2_test_avg':Adj_r2_test_avg}, ignore_index=True)

        ar= ar.append({'Features': l, 'P_Values': b},ignore_index=True)
        c.append(l)
        print(l,':',b)
print(c)
ar.head(30)

Met_Color : 0.5624870617959193
cc : 0.18775272767856152
ABS : 0.13051100675397756
Airbag_1 : 0.25558144942085276
Airbag_2 : 0.7410785224122204
Central_Lock : 0.29528576725629285
Power_Steering : 0.19131862179544454
Radio : 0.29229484486244367
Mistlamps : 0.9207132387975276
Backseat_Divider : 0.4504677423964323
Metallic_Rim : 0.13437338548564226
Radio_cassette : 0.27129803274817754
['Met_Color', 'cc', 'ABS', 'Airbag_1', 'Airbag_2', 'Central_Lock', 'Power_Steering', 'Radio', 'Mistlamps', 'Backseat_Divider', 'Metallic_Rim', 'Radio_cassette']


,Features,P_Values
0,Met_Color,0.562487
1,cc,0.187753
2,ABS,0.130511
3,Airbag_1,0.255581
4,Airbag_2,0.741079
5,Central_Lock,0.295286
6,Power_Steering,0.191319
7,Radio,0.292295
8,Mistlamps,0.920713
9,Backseat_Divider,0.450468


In [26]:
X2= X.copy()
X2.head()

X2.drop( columns = c, axis=1,inplace=True)

X2.head()

,Id,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Automatic,Doors,Cylinders,Gears,...,Mfr_Guarantee,BOVAG_Guarantee,Guarantee_Period,Airco,Automatic_airco,Boardcomputer,CD_Player,Powered_Windows,Sport_Model,Tow_Bar
0,1,23,10,2002,46986,90,0,3,4,5,...,0,1,3,0,0,1,0,1,0,0
1,2,23,10,2002,72937,90,0,3,4,5,...,0,1,3,1,0,1,1,0,0,0
2,3,24,9,2002,41711,90,0,3,4,5,...,1,1,3,0,0,1,0,0,0,0
3,4,26,7,2002,48000,90,0,3,4,5,...,1,1,3,0,0,1,0,0,0,0
4,5,30,3,2002,38500,90,0,3,4,5,...,1,1,3,1,0,1,0,1,0,0


In [27]:
p= len(X2.columns)
print(p)
n= len(X2.index)
print(n)

22
1436


In [28]:
# for X2

reg = LinearRegression().fit(X2, y)
print(reg.score(X2, y))
print(reg.coef_)
print(reg.intercept_)

y_pred = reg.predict(X2)
# print(y_pred)
print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
print("R2: %.2f" % r2_score(y, y_pred))

R2_t_fulldata= reg.score(X2, y)
coefs_fulldata = reg.coef_
intercep_fulldata = reg.intercept_

y_pred = reg.predict(X2)
mse = mean_squared_error(y, y_pred)
r2= r2_score(y, y_pred)
# print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_test))
# print("R2: %.3f" % r2_score(y_test, y_pred_test))
Adj_r2 = 1-(1-r2)*((n-1)/(n-p-1))

# result_fulldata = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

case_fulldata = 'Removed feature with P > 0.05'

result_fulldata=result_fulldata.append({'case': case_fulldata, 'R2 score': R2_t_fulldata, 'intercepts': intercep_fulldata, 'coffeients': coefs_fulldata,'Mean squared error': mse ,'R2' : r2, 'Adj_r2': Adj_r2}, ignore_index=True)


result_fulldata.head()


0.9073943451053001
[ 1.78336757e+00 -1.57635185e+02 -1.34676379e+02  2.43592970e+01
 -1.30830013e-02  2.51713087e+01  3.68558514e+02  5.79987474e+01
  0.00000000e+00  4.00382896e+02  7.31408161e+00  8.88902712e+00
  2.70637158e+02  4.78830200e+02  6.32963717e+01  1.71080379e+02
  2.36283593e+03 -2.68493881e+02  1.88499109e+02  3.72817673e+02
  1.74030873e+02 -1.91488927e+02]
-44966.623978890384
Mean squared error: 1217367.21
R2: 0.91


,case,R2 score,intercepts,coffeients,Mean squared error,R2,Adj_r2
0,All features,0.908355,-2.197033e+13,"[1.6060105730085568, 913261203.5167035, 913261...",1.204742e+06,0.908355,0.906131
1,Removed feature with P > 0.05,0.907394,-4.496662e+04,"[1.7833675684437138, -157.6351853482074, -134....",1.217367e+06,0.907394,0.905953


In [29]:
i=0

result = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'MSE_Train','R2_Train', 'Adj_r2_train' ,
                                'MSE_Test','R2_Test', 'Adj_r2_test' ])

while i < 5:

    # print('Simulation number :', i)


    # repeat this activity multiple times and record the R2 and MSE for each run
    X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.3)

    reg2 = LinearRegression().fit(X_train, y_train)
    r2_t= reg2.score(X2, y)
    coefs = reg2.coef_
    intercep = reg2.intercept_
    # print(reg2.score(X2, y))
    # print(reg2.coef_)
    # print(reg2.intercept_)

    y_pred_train = reg2.predict(X_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))






    y_pred_test = reg2.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))





    i= i+1

    result=result.append({'R2 score': r2_t, 'intercepts': intercep, 'coffeients': coefs, 'MSE_Train': mse_train,
                             'R2_Train': r2_train, 'Adj_r2_train': Adj_r2_train, 
                             'MSE_Test': mse_test, 'R2_Test': r2_test, 
                             'Adj_r2_test': Adj_r2_test }, ignore_index=True)





r2_train_avg = result['R2_Train'].mean()
Adj_r2_train_avg= result['Adj_r2_train'].mean()
mse_train_avg = result['MSE_Train'].mean()



r2_test_avg = result['R2_Test'].mean()
Adj_r2_test_avg= result['Adj_r2_test'].mean()
mse_test_avg = result['MSE_Test'].mean()

case= 'Removed feature with P > 0.05'

# result_avg = pd.DataFrame( columns = ['Case','Average MSE_Train', 'Average R2_Train',
#                                         'Adj_r2_train_avg', 'Average MSE_Test', 'Average R2_Test',
#                                          'Adj_r2_test_avg'])

result_avg= result_avg.append({'Case': case, 'Average MSE_Train': mse_train_avg, 'Average R2_Train': r2_train_avg,
                                'Adj_r2_train_avg': Adj_r2_train_avg, 'Average MSE_Test': mse_test_avg, 
                                'Average R2_Test': r2_test_avg, 'Adj_r2_test_avg':Adj_r2_test_avg}, ignore_index=True)

result.head(12)

# print(' The total Mean of results after', i , ' number of simulations is :')

# result_avg.head()
    # df_f=df_f.append({'Name':name_4s,'Contact':c,'address':d,'url':u,'ratings':rat,'comments':com,'image':image}, ignore_index=True)

,R2 score,intercepts,coffeients,MSE_Train,R2_Train,Adj_r2_train,MSE_Test,R2_Test,Adj_r2_test
0,0.906552,-41946.639082,"[1.7694869510728743, -154.06459494721648, -128...",1.118762e+06,0.913941,0.912601,1.484177e+06,0.889759,0.888042
1,0.906765,-40709.059889,"[1.4416306884264338, -146.90194918146193, -123...",1.186094e+06,0.910466,0.909072,1.317866e+06,0.897814,0.896223
2,0.906388,-46510.347835,"[1.7274675651695208, -157.4516557739675, -136....",1.121989e+06,0.907414,0.905973,1.483853e+06,0.904324,0.902835
3,0.899695,-45387.923524,"[1.0219159284467252, -129.15525505099225, -109...",1.054399e+06,0.918637,0.917370,1.934601e+06,0.857526,0.855308
4,0.906836,-44826.679826,"[1.9359472229452557, -162.79937839702774, -136...",1.290443e+06,0.908049,0.906617,1.071410e+06,0.902446,0.900927


In [30]:
print(' The total Mean of results after', i , ' number of simulations is :')

result_avg.head()

 The total Mean of results after 5  number of simulations is :


,Case,Average MSE_Train,Average R2_Train,Adj_r2_train_avg,Average MSE_Test,Average R2_Test,Adj_r2_test_avg
0,All features,1.201881e+06,0.911860,0.909721,2.269577e+06,0.820204,0.815840
1,Removed feature with P > 0.05,1.154337e+06,0.911701,0.910327,1.458381e+06,0.890374,0.888667


In [31]:
X5 = X2.copy()
X5.head()

,Id,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Automatic,Doors,Cylinders,Gears,...,Mfr_Guarantee,BOVAG_Guarantee,Guarantee_Period,Airco,Automatic_airco,Boardcomputer,CD_Player,Powered_Windows,Sport_Model,Tow_Bar
0,1,23,10,2002,46986,90,0,3,4,5,...,0,1,3,0,0,1,0,1,0,0
1,2,23,10,2002,72937,90,0,3,4,5,...,0,1,3,1,0,1,1,0,0,0
2,3,24,9,2002,41711,90,0,3,4,5,...,1,1,3,0,0,1,0,0,0,0
3,4,26,7,2002,48000,90,0,3,4,5,...,1,1,3,0,0,1,0,0,0,0
4,5,30,3,2002,38500,90,0,3,4,5,...,1,1,3,1,0,1,0,1,0,0


In [32]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='forward',n_features_to_select=8)
sfs.fit(X5, y)
f5_forward_8= sfs.get_feature_names_out()
print(sfs.get_feature_names_out())

X5_forward_8=X5[ f5_forward_8]
X5_forward_8.head()

p= len(X5_forward_8.columns)
print(p)
n= len(X5_forward_8.index)
print(n)

i=0

result = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'MSE_Train','R2_Train', 'Adj_r2_train' ,
                                'MSE_Test','R2_Test', 'Adj_r2_test' ])

while i < 5:

    # print('Simulation number :', i)


    # repeat this activity multiple times and record the R2 and MSE for each run
    X_train, X_test, y_train, y_test = train_test_split(X5_forward_8, y, test_size = 0.3)

    reg2 = LinearRegression().fit(X_train, y_train)
    r2_t= reg2.score(X5_forward_8, y)
    coefs = reg2.coef_
    intercep = reg2.intercept_
    # print(reg2.score(X5_forward_8, y))
    # print(reg2.coef_)
    # print(reg2.intercept_)

    y_pred_train = reg2.predict(X_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))






    y_pred_test = reg2.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))





    i= i+1

    result=result.append({'R2 score': r2_t, 'intercepts': intercep, 'coffeients': coefs, 'MSE_Train': mse_train,
                             'R2_Train': r2_train, 'Adj_r2_train': Adj_r2_train, 
                             'MSE_Test': mse_test, 'R2_Test': r2_test, 
                             'Adj_r2_test': Adj_r2_test }, ignore_index=True)





r2_train_avg = result['R2_Train'].mean()
Adj_r2_train_avg= result['Adj_r2_train'].mean()
mse_train_avg = result['MSE_Train'].mean()



r2_test_avg = result['R2_Test'].mean()
Adj_r2_test_avg= result['Adj_r2_test'].mean()
mse_test_avg = result['MSE_Test'].mean()

case= 'Serial feature selection forward 8 features'

# result_avg = pd.DataFrame( columns = ['Case','Average MSE_Train', 'Average R2_Train',
#                                         'Adj_r2_train_avg', 'Average MSE_Test', 'Average R2_Test',
#                                          'Adj_r2_test_avg'])

result_avg= result_avg.append({'Case': case, 'Average MSE_Train': mse_train_avg, 'Average R2_Train': r2_train_avg,
                                'Adj_r2_train_avg': Adj_r2_train_avg, 'Average MSE_Test': mse_test_avg, 
                                'Average R2_Test': r2_test_avg, 'Adj_r2_test_avg':Adj_r2_test_avg}, ignore_index=True)

result.head(12)

# print(' The total Mean of results after', i , ' number of simulations is :')

# result_avg.head()
    # df_f=df_f.append({'Name':name_8s,'Contact':c,'address':d,'url':u,'ratings':rat,'comments':com,'image':image}, ignore_index=True)





['Id' 'Mfg_Year' 'Weight' 'Mfr_Guarantee' 'Guarantee_Period' 'Airco'
 'Automatic_airco' 'Powered_Windows']
8
1436


,R2 score,intercepts,coffeients,MSE_Train,R2_Train,Adj_r2_train,MSE_Test,R2_Test,Adj_r2_test
0,0.883753,-5.013429e+06,"[3.5125155756181927, 2504.8754434694933, 11.03...",1.620168e+06,0.882009,0.881347,1.313572e+06,0.888491,0.887866
1,0.883882,-4.829221e+06,"[3.279103671039708, 2412.8650489010283, 10.959...",1.459931e+06,0.884246,0.883597,1.681589e+06,0.883130,0.882475
2,0.883959,-4.852928e+06,"[3.272645698430805, 2424.8843177212652, 10.731...",1.472023e+06,0.887815,0.887187,1.650005e+06,0.874972,0.874271
3,0.883708,-4.907643e+06,"[3.365798548836268, 2451.8750583672822, 11.286...",1.528438e+06,0.884860,0.884215,1.529444e+06,0.880854,0.880187
4,0.883916,-4.873799e+06,"[3.312274390199082, 2435.9633771597587, 9.5000...",1.579255e+06,0.881770,0.881107,1.401835e+06,0.888947,0.888324


In [33]:
print(' The total Mean of results after', i , ' number of simulations is :')

result_avg.head()

 The total Mean of results after 5  number of simulations is :


,Case,Average MSE_Train,Average R2_Train,Adj_r2_train_avg,Average MSE_Test,Average R2_Test,Adj_r2_test_avg
0,All features,1.201881e+06,0.911860,0.909721,2.269577e+06,0.820204,0.815840
1,Removed feature with P > 0.05,1.154337e+06,0.911701,0.910327,1.458381e+06,0.890374,0.888667
2,Serial feature selection forward 8 features,1.531963e+06,0.884140,0.883491,1.515289e+06,0.883279,0.882624


In [34]:
# for X5_forward_8

reg = LinearRegression().fit(X5_forward_8, y)
print(reg.score(X5_forward_8, y))
print(reg.coef_)
print(reg.intercept_)

y_pred = reg.predict(X5_forward_8)
# print(y_pred)
print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
print("R2: %.2f" % r2_score(y, y_pred))

R2_t_fulldata= reg.score(X5_forward_8, y)
coefs_fulldata = reg.coef_
intercep_fulldata = reg.intercept_

y_pred = reg.predict(X5_forward_8)
mse = mean_squared_error(y, y_pred)
r2= r2_score(y, y_pred)
# print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_test))
# print("R2: %.3f" % r2_score(y_test, y_pred_test))
Adj_r2 = 1-(1-r2)*((n-1)/(n-p-1))

# result_fulldata = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

case_fulldata = 'Serial feature selection forward 8 features'

result_fulldata=result_fulldata.append({'case': case_fulldata, 'R2 score': R2_t_fulldata, 'intercepts': intercep_fulldata, 'coffeients': coefs_fulldata,'Mean squared error': mse ,'R2' : r2, 'Adj_r2': Adj_r2}, ignore_index=True)


result_fulldata.head()


0.8841726083680768
[   3.28483619 2439.99169137   10.36480335  548.50086179   45.53052425
  284.14267012 2628.64909896  456.40366624]
-4882771.956703732
Mean squared error: 1522633.45
R2: 0.88


,case,R2 score,intercepts,coffeients,Mean squared error,R2,Adj_r2
0,All features,0.908355,-2.197033e+13,"[1.6060105730085568, 913261203.5167035, 913261...",1.204742e+06,0.908355,0.906131
1,Removed feature with P > 0.05,0.907394,-4.496662e+04,"[1.7833675684437138, -157.6351853482074, -134....",1.217367e+06,0.907394,0.905953
2,Serial feature selection forward 8 features,0.884173,-4.882772e+06,"[3.284836191751551, 2439.9916913737075, 10.364...",1.522633e+06,0.884173,0.883523


In [35]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='backward',n_features_to_select=8)
sfs.fit(X5, y)
f5_backward_8= sfs.get_feature_names_out()
print(sfs.get_feature_names_out())

X5_backward_8=X5[ f5_backward_8]
X5_backward_8.head()

p= len(X5_backward_8.columns)
print(p)
n= len(X5_backward_8.index)
print(n)


i=0

result = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'MSE_Train','R2_Train', 'Adj_r2_train' ,
                                'MSE_Test','R2_Test', 'Adj_r2_test' ])

while i < 5:

    # print('Simulation number :', i)


    # repeat this activity multiple times and record the R2 and MSE for each run
    X_train, X_test, y_train, y_test = train_test_split(X5_backward_8, y, test_size = 0.3)

    reg2 = LinearRegression().fit(X_train, y_train)
    r2_t= reg2.score(X5_backward_8, y)
    coefs = reg2.coef_
    intercep = reg2.intercept_
    # print(reg2.score(X5_backward_8, y))
    # print(reg2.coef_)
    # print(reg2.intercept_)

    y_pred_train = reg2.predict(X_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))






    y_pred_test = reg2.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))





    i= i+1

    result=result.append({'R2 score': r2_t, 'intercepts': intercep, 'coffeients': coefs, 'MSE_Train': mse_train,
                             'R2_Train': r2_train, 'Adj_r2_train': Adj_r2_train, 
                             'MSE_Test': mse_test, 'R2_Test': r2_test, 
                             'Adj_r2_test': Adj_r2_test }, ignore_index=True)





r2_train_avg = result['R2_Train'].mean()
Adj_r2_train_avg= result['Adj_r2_train'].mean()
mse_train_avg = result['MSE_Train'].mean()



r2_test_avg = result['R2_Test'].mean()
Adj_r2_test_avg= result['Adj_r2_test'].mean()
mse_test_avg = result['MSE_Test'].mean()

case= 'Serial feature selection backward 8 features'

# result_avg = pd.DataFrame( columns = ['Case','Average MSE_Train', 'Average R2_Train',
#                                         'Adj_r2_train_avg', 'Average MSE_Test', 'Average R2_Test',
#                                          'Adj_r2_test_avg'])

result_avg= result_avg.append({'Case': case, 'Average MSE_Train': mse_train_avg, 'Average R2_Train': r2_train_avg,
                                'Adj_r2_train_avg': Adj_r2_train_avg, 'Average MSE_Test': mse_test_avg, 
                                'Average R2_Test': r2_test_avg, 'Adj_r2_test_avg':Adj_r2_test_avg}, ignore_index=True)

result.head(12)

# print(' The total Mean of results after', i , ' number of simulations is :')

# result_avg.head()
    # df_f=df_f.append({'Name':name_8s,'Contact':c,'address':d,'url':u,'ratings':rat,'comments':com,'image':image}, ignore_index=True)





['Id' 'Age_08_04' 'Mfg_Month' 'Automatic' 'Doors' 'Mfr_Guarantee'
 'Automatic_airco' 'Powered_Windows']
8
1436


,R2 score,intercepts,coffeients,MSE_Train,R2_Train,Adj_r2_train,MSE_Test,R2_Test,Adj_r2_test
0,0.869912,20430.220838,"[3.5688075509898667, -223.54925907467018, -176...",1.769537e+06,0.868059,0.867319,1.571500e+06,0.874527,0.873824
1,0.869209,20438.279134,"[3.208619993637456, -220.6717345119838, -187.6...",1.624325e+06,0.877407,0.876719,1.940887e+06,0.849552,0.848708
2,0.869066,20450.416348,"[3.3566886644696963, -222.0467859333749, -203....",1.604403e+06,0.878232,0.877549,1.993601e+06,0.847300,0.846444
3,0.869893,19791.139364,"[3.3723137117209077, -218.52297465812492, -178...",1.685048e+06,0.876991,0.876301,1.769350e+06,0.850698,0.849861
4,0.869866,19777.615064,"[2.932834337157119, -207.06039276174576, -165....",1.735183e+06,0.872017,0.871300,1.653613e+06,0.861282,0.860504


In [36]:
print(' The total Mean of results after', i , ' number of simulations is :')

result_avg.head()

 The total Mean of results after 5  number of simulations is :


,Case,Average MSE_Train,Average R2_Train,Adj_r2_train_avg,Average MSE_Test,Average R2_Test,Adj_r2_test_avg
0,All features,1.201881e+06,0.911860,0.909721,2.269577e+06,0.820204,0.815840
1,Removed feature with P > 0.05,1.154337e+06,0.911701,0.910327,1.458381e+06,0.890374,0.888667
2,Serial feature selection forward 8 features,1.531963e+06,0.884140,0.883491,1.515289e+06,0.883279,0.882624
3,Serial feature selection backward 8 features,1.683699e+06,0.874541,0.873838,1.785790e+06,0.856672,0.855868


In [37]:
# for X5_backward_8

reg = LinearRegression().fit(X5_backward_8, y)
print(reg.score(X5_backward_8, y))
print(reg.coef_)
print(reg.intercept_)

y_pred = reg.predict(X5_backward_8)
# print(y_pred)
print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
print("R2: %.2f" % r2_score(y, y_pred))

R2_t_fulldata= reg.score(X5_backward_8, y)
coefs_fulldata = reg.coef_
intercep_fulldata = reg.intercept_

y_pred = reg.predict(X5_backward_8)
mse = mean_squared_error(y, y_pred)
r2= r2_score(y, y_pred)
# print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_test))
# print("R2: %.3f" % r2_score(y_test, y_pred_test))
Adj_r2 = 1-(1-r2)*((n-1)/(n-p-1))

# result_fulldata = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

case_fulldata = 'Serial feature selection backward 8 features'

result_fulldata=result_fulldata.append({'case': case_fulldata, 'R2 score': R2_t_fulldata, 'intercepts': intercep_fulldata, 'coffeients': coefs_fulldata,'Mean squared error': mse ,'R2' : r2, 'Adj_r2': Adj_r2}, ignore_index=True)


result_fulldata.head()


0.8703320407529368
[   3.25468519 -216.34524497 -176.08462985  526.64854475  174.83716681
  396.32468246 3204.56003028  625.87278378]
20034.021676036624
Mean squared error: 1704577.56
R2: 0.87


,case,R2 score,intercepts,coffeients,Mean squared error,R2,Adj_r2
0,All features,0.908355,-2.197033e+13,"[1.6060105730085568, 913261203.5167035, 913261...",1.204742e+06,0.908355,0.906131
1,Removed feature with P > 0.05,0.907394,-4.496662e+04,"[1.7833675684437138, -157.6351853482074, -134....",1.217367e+06,0.907394,0.905953
2,Serial feature selection forward 8 features,0.884173,-4.882772e+06,"[3.284836191751551, 2439.9916913737075, 10.364...",1.522633e+06,0.884173,0.883523
3,Serial feature selection backward 8 features,0.870332,2.003402e+04,"[3.2546851851304592, -216.3452449689577, -176....",1.704578e+06,0.870332,0.869605


In [38]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='forward',n_features_to_select=4)
sfs.fit(X5, y)
f5_forward_4= sfs.get_feature_names_out()
print(sfs.get_feature_names_out())

['Id' 'Mfg_Year' 'Airco' 'Automatic_airco']


In [39]:
X5_forward_4=X5[ f5_forward_4]
X5_forward_4.head()

,Id,Mfg_Year,Airco,Automatic_airco
0,1,2002,0,0
1,2,2002,1,0
2,3,2002,0,0
3,4,2002,0,0
4,5,2002,1,0


In [40]:
X5_forward_4.shape

(1436, 4)

In [41]:
p= len(X5_forward_4.columns)
print(p)
n= len(X5_forward_4.index)
print(n)



i=0

result = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'MSE_Train','R2_Train', 'Adj_r2_train' ,
                                'MSE_Test','R2_Test', 'Adj_r2_test' ])

while i < 5:

    # print('Simulation number :', i)


    # repeat this activity multiple times and record the R2 and MSE for each run
    X_train, X_test, y_train, y_test = train_test_split(X5_forward_4, y, test_size = 0.3)

    reg2 = LinearRegression().fit(X_train, y_train)
    r2_t= reg2.score(X5_forward_4, y)
    coefs = reg2.coef_
    intercep = reg2.intercept_
    # print(reg2.score(X5_forward_4, y))
    # print(reg2.coef_)
    # print(reg2.intercept_)

    y_pred_train = reg2.predict(X_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))






    y_pred_test = reg2.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))





    i= i+1

    result=result.append({'R2 score': r2_t, 'intercepts': intercep, 'coffeients': coefs, 'MSE_Train': mse_train,
                             'R2_Train': r2_train, 'Adj_r2_train': Adj_r2_train, 
                             'MSE_Test': mse_test, 'R2_Test': r2_test, 
                             'Adj_r2_test': Adj_r2_test }, ignore_index=True)





r2_train_avg = result['R2_Train'].mean()
Adj_r2_train_avg= result['Adj_r2_train'].mean()
mse_train_avg = result['MSE_Train'].mean()



r2_test_avg = result['R2_Test'].mean()
Adj_r2_test_avg= result['Adj_r2_test'].mean()
mse_test_avg = result['MSE_Test'].mean()

case= 'Serial feature selection Forward 4 features'

# result_avg = pd.DataFrame( columns = ['Case','Average MSE_Train', 'Average R2_Train',
#                                         'Adj_r2_train_avg', 'Average MSE_Test', 'Average R2_Test',
#                                          'Adj_r2_test_avg'])

result_avg= result_avg.append({'Case': case, 'Average MSE_Train': mse_train_avg, 'Average R2_Train': r2_train_avg,
                                'Adj_r2_train_avg': Adj_r2_train_avg, 'Average MSE_Test': mse_test_avg, 
                                'Average R2_Test': r2_test_avg, 'Adj_r2_test_avg':Adj_r2_test_avg}, ignore_index=True)

result.head(12)

# print(' The total Mean of results after', i , ' number of simulations is :')

# result_avg.head()
    # df_f=df_f.append({'Name':name_4s,'Contact':c,'address':d,'url':u,'ratings':rat,'comments':com,'image':image}, ignore_index=True)

4
1436


,R2 score,intercepts,coffeients,MSE_Train,R2_Train,Adj_r2_train,MSE_Test,R2_Test,Adj_r2_test
0,0.861616,-5.354825e+06,"[3.667184413225571, 2681.7141320165774, 619.51...",1.851360e+06,0.854308,0.853900,1.744055e+06,0.876775,0.876430
1,0.861786,-5.300090e+06,"[3.539011517931851, 2654.3529616604533, 681.66...",1.838657e+06,0.861842,0.861456,1.766219e+06,0.861625,0.861238
2,0.861390,-5.343760e+06,"[3.4772502892793606, 2676.249310741175, 653.16...",1.841913e+06,0.861558,0.861171,1.776009e+06,0.860863,0.860474
3,0.861680,-5.174448e+06,"[3.260912874355679, 2591.6562373485617, 568.83...",1.723434e+06,0.858574,0.858179,2.039559e+06,0.866591,0.866218
4,0.861426,-5.078268e+06,"[3.140609390663718, 2543.581476570061, 652.246...",1.819268e+06,0.865859,0.865484,1.827234e+06,0.848002,0.847577


In [42]:
print(' The total Mean of results after', i , ' number of simulations is :')

result_avg.head()

 The total Mean of results after 5  number of simulations is :


,Case,Average MSE_Train,Average R2_Train,Adj_r2_train_avg,Average MSE_Test,Average R2_Test,Adj_r2_test_avg
0,All features,1.201881e+06,0.911860,0.909721,2.269577e+06,0.820204,0.815840
1,Removed feature with P > 0.05,1.154337e+06,0.911701,0.910327,1.458381e+06,0.890374,0.888667
2,Serial feature selection forward 8 features,1.531963e+06,0.884140,0.883491,1.515289e+06,0.883279,0.882624
3,Serial feature selection backward 8 features,1.683699e+06,0.874541,0.873838,1.785790e+06,0.856672,0.855868
4,Serial feature selection Forward 4 features,1.814926e+06,0.860428,0.860038,1.830615e+06,0.862771,0.862388


In [43]:
# for X5_forward_4

reg = LinearRegression().fit(X5_forward_4, y)
print(reg.score(X5_forward_4, y))
print(reg.coef_)
print(reg.intercept_)

y_pred = reg.predict(X5_forward_4)
# print(y_pred)
print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
print("R2: %.2f" % r2_score(y, y_pred))

R2_t_fulldata= reg.score(X5_forward_4, y)
coefs_fulldata = reg.coef_
intercep_fulldata = reg.intercept_

y_pred = reg.predict(X5_forward_4)
mse = mean_squared_error(y, y_pred)
r2= r2_score(y, y_pred)
# print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_test))
# print("R2: %.3f" % r2_score(y_test, y_pred_test))
Adj_r2 = 1-(1-r2)*((n-1)/(n-p-1))

# result_fulldata = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

case_fulldata = 'Serial feature selection forward 4 features'

result_fulldata=result_fulldata.append({'case': case_fulldata, 'R2 score': R2_t_fulldata, 'intercepts': intercep_fulldata, 'coffeients': coefs_fulldata,'Mean squared error': mse ,'R2' : r2, 'Adj_r2': Adj_r2}, ignore_index=True)


result_fulldata.head()


0.8619144333570951
[   3.51626623 2665.80568998  639.45619285 3199.70650297]
-5322924.542626247
Mean squared error: 1815233.00
R2: 0.86


,case,R2 score,intercepts,coffeients,Mean squared error,R2,Adj_r2
0,All features,0.908355,-2.197033e+13,"[1.6060105730085568, 913261203.5167035, 913261...",1.204742e+06,0.908355,0.906131
1,Removed feature with P > 0.05,0.907394,-4.496662e+04,"[1.7833675684437138, -157.6351853482074, -134....",1.217367e+06,0.907394,0.905953
2,Serial feature selection forward 8 features,0.884173,-4.882772e+06,"[3.284836191751551, 2439.9916913737075, 10.364...",1.522633e+06,0.884173,0.883523
3,Serial feature selection backward 8 features,0.870332,2.003402e+04,"[3.2546851851304592, -216.3452449689577, -176....",1.704578e+06,0.870332,0.869605
4,Serial feature selection forward 4 features,0.861914,-5.322925e+06,"[3.5162662254997987, 2665.805689981417, 639.45...",1.815233e+06,0.861914,0.861528


In [44]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='backward',n_features_to_select=4)
sfs.fit(X5, y)
f5_backward_4= sfs.get_feature_names_out()
print(sfs.get_feature_names_out())

X5_backward_4=X5[ f5_backward_4]
X5_backward_4.head()

p= len(X5_backward_4.columns)
print(p)
n= len(X5_backward_4.index)
print(n)


i=0

result = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'MSE_Train','R2_Train', 'Adj_r2_train' ,
                                'MSE_Test','R2_Test', 'Adj_r2_test' ])

while i < 5:

    # print('Simulation number :', i)


    # repeat this activity multiple times and record the R2 and MSE for each run
    X_train, X_test, y_train, y_test = train_test_split(X5_backward_4, y, test_size = 0.3)

    reg2 = LinearRegression().fit(X_train, y_train)
    r2_t= reg2.score(X5_backward_4, y)
    coefs = reg2.coef_
    intercep = reg2.intercept_
    # print(reg2.score(X5_backward_4, y))
    # print(reg2.coef_)
    # print(reg2.intercept_)

    y_pred_train = reg2.predict(X_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))






    y_pred_test = reg2.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))





    i= i+1

    result=result.append({'R2 score': r2_t, 'intercepts': intercep, 'coffeients': coefs, 'MSE_Train': mse_train,
                             'R2_Train': r2_train, 'Adj_r2_train': Adj_r2_train, 
                             'MSE_Test': mse_test, 'R2_Test': r2_test, 
                             'Adj_r2_test': Adj_r2_test }, ignore_index=True)





r2_train_avg = result['R2_Train'].mean()
Adj_r2_train_avg= result['Adj_r2_train'].mean()
mse_train_avg = result['MSE_Train'].mean()



r2_test_avg = result['R2_Test'].mean()
Adj_r2_test_avg= result['Adj_r2_test'].mean()
mse_test_avg = result['MSE_Test'].mean()

case= 'Serial feature selection backward 4 features'

# result_avg = pd.DataFrame( columns = ['Case','Average MSE_Train', 'Average R2_Train',
#                                         'Adj_r2_train_avg', 'Average MSE_Test', 'Average R2_Test',
#                                          'Adj_r2_test_avg'])

result_avg= result_avg.append({'Case': case, 'Average MSE_Train': mse_train_avg, 'Average R2_Train': r2_train_avg,
                                'Adj_r2_train_avg': Adj_r2_train_avg, 'Average MSE_Test': mse_test_avg, 
                                'Average R2_Test': r2_test_avg, 'Adj_r2_test_avg':Adj_r2_test_avg}, ignore_index=True)

result.head(12)

# print(' The total Mean of results after', i , ' number of simulations is :')

# result_avg.head()
    # df_f=df_f.append({'Name':name_4s,'Contact':c,'address':d,'url':u,'ratings':rat,'comments':com,'image':image}, ignore_index=True)





['Id' 'Age_08_04' 'Mfg_Month' 'Automatic_airco']
4
1436


,R2 score,intercepts,coffeients,MSE_Train,R2_Train,Adj_r2_train,MSE_Test,R2_Test,Adj_r2_test
0,0.856717,21739.952520,"[3.3302348479244244, -224.5582668341899, -183....",1.819425e+06,0.855745,0.855342,2.033119e+06,0.858702,0.858307
1,0.856650,21405.761572,"[2.9905395766132137, -215.27999339524888, -172...",1.676773e+06,0.865339,0.864963,2.368652e+06,0.839083,0.838633
2,0.856451,21357.646220,"[3.275193310255124, -219.02978317394306, -174....",1.912155e+06,0.857248,0.856849,1.828525e+06,0.854389,0.853982
3,0.856828,21954.648975,"[3.701188537497408, -233.6429589351149, -180.5...",2.012604e+06,0.852411,0.851998,1.577792e+06,0.868386,0.868018
4,0.856789,21862.776946,"[3.327175271439206, -227.96084867997536, -171....",1.915333e+06,0.861810,0.861424,1.806313e+06,0.842611,0.842172


In [45]:
print(' The total Mean of results after', i , ' number of simulations is :')

result_avg.head(10)

 The total Mean of results after 5  number of simulations is :


,Case,Average MSE_Train,Average R2_Train,Adj_r2_train_avg,Average MSE_Test,Average R2_Test,Adj_r2_test_avg
0,All features,1.201881e+06,0.911860,0.909721,2.269577e+06,0.820204,0.815840
1,Removed feature with P > 0.05,1.154337e+06,0.911701,0.910327,1.458381e+06,0.890374,0.888667
2,Serial feature selection forward 8 features,1.531963e+06,0.884140,0.883491,1.515289e+06,0.883279,0.882624
3,Serial feature selection backward 8 features,1.683699e+06,0.874541,0.873838,1.785790e+06,0.856672,0.855868
4,Serial feature selection Forward 4 features,1.814926e+06,0.860428,0.860038,1.830615e+06,0.862771,0.862388
5,Serial feature selection backward 4 features,1.867258e+06,0.858511,0.858115,1.922880e+06,0.852634,0.852222


In [46]:
# for X5_backward_4

reg = LinearRegression().fit(X5_backward_4, y)
print(reg.score(X5_backward_4, y))
print(reg.coef_)
print(reg.intercept_)

y_pred = reg.predict(X5_backward_4)
# print(y_pred)
print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
print("R2: %.2f" % r2_score(y, y_pred))

R2_t_fulldata= reg.score(X5_backward_4, y)
coefs_fulldata = reg.coef_
intercep_fulldata = reg.intercept_

y_pred = reg.predict(X5_backward_4)
mse = mean_squared_error(y, y_pred)
r2= r2_score(y, y_pred)
# print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_test))
# print("R2: %.3f" % r2_score(y_test, y_pred_test))
Adj_r2 = 1-(1-r2)*((n-1)/(n-p-1))

# result_fulldata = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

case_fulldata = 'Serial feature selection backward 8 features'

result_fulldata=result_fulldata.append({'case': case_fulldata, 'R2 score': R2_t_fulldata, 'intercepts': intercep_fulldata, 'coffeients': coefs_fulldata,'Mean squared error': mse ,'R2' : r2, 'Adj_r2': Adj_r2}, ignore_index=True)


result_fulldata.head(10)


0.8571537713569309
[   3.38403147 -225.86612171 -181.05562132 3356.36829474]
21740.918674312776
Mean squared error: 1877815.29
R2: 0.86


,case,R2 score,intercepts,coffeients,Mean squared error,R2,Adj_r2
0,All features,0.908355,-2.197033e+13,"[1.6060105730085568, 913261203.5167035, 913261...",1.204742e+06,0.908355,0.906131
1,Removed feature with P > 0.05,0.907394,-4.496662e+04,"[1.7833675684437138, -157.6351853482074, -134....",1.217367e+06,0.907394,0.905953
2,Serial feature selection forward 8 features,0.884173,-4.882772e+06,"[3.284836191751551, 2439.9916913737075, 10.364...",1.522633e+06,0.884173,0.883523
3,Serial feature selection backward 8 features,0.870332,2.003402e+04,"[3.2546851851304592, -216.3452449689577, -176....",1.704578e+06,0.870332,0.869605
4,Serial feature selection forward 4 features,0.861914,-5.322925e+06,"[3.5162662254997987, 2665.805689981417, 639.45...",1.815233e+06,0.861914,0.861528
5,Serial feature selection backward 8 features,0.857154,2.174092e+04,"[3.384031470524448, -225.86612170543316, -181....",1.877815e+06,0.857154,0.856754


In [47]:
X.shape

(1436, 34)

In [48]:
from sklearn.neighbors import KNeighborsRegressor

p= len(X.columns)
print(p)
n= len(X.index)
print(n)


neigh = KNeighborsRegressor(n_neighbors=3)

knn_reg = neigh.fit(X, y)
R2_t_fulldata= knn_reg.score(X, y)
#print(reg.coef_)
#print(reg.intercept_)
y_pred = knn_reg.predict(X)
# print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
# print("Coefficient of determination: %.3f" % r2_score(y, y_pred))




mse = mean_squared_error(y, y_pred)
r2= r2_score(y, y_pred)

Adj_r2 = 1-(1-r2)*((n-1)/(n-p-1))

# result_fulldata = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

case_fulldata = 'knn k =3'

result_fulldata=result_fulldata.append({'case': case_fulldata, 'R2 score': R2_t_fulldata, 'Mean squared error': mse ,'R2' : r2, 'Adj_r2': Adj_r2}, ignore_index=True)


result_fulldata.head(15)


34
1436


,case,R2 score,intercepts,coffeients,Mean squared error,R2,Adj_r2
0,All features,0.908355,-2.197033e+13,"[1.6060105730085568, 913261203.5167035, 913261...",1.204742e+06,0.908355,0.906131
1,Removed feature with P > 0.05,0.907394,-4.496662e+04,"[1.7833675684437138, -157.6351853482074, -134....",1.217367e+06,0.907394,0.905953
2,Serial feature selection forward 8 features,0.884173,-4.882772e+06,"[3.284836191751551, 2439.9916913737075, 10.364...",1.522633e+06,0.884173,0.883523
3,Serial feature selection backward 8 features,0.870332,2.003402e+04,"[3.2546851851304592, -216.3452449689577, -176....",1.704578e+06,0.870332,0.869605
4,Serial feature selection forward 4 features,0.861914,-5.322925e+06,"[3.5162662254997987, 2665.805689981417, 639.45...",1.815233e+06,0.861914,0.861528
5,Serial feature selection backward 8 features,0.857154,2.174092e+04,"[3.384031470524448, -225.86612170543316, -181....",1.877815e+06,0.857154,0.856754
6,knn k =3,0.853746,NaN,NaN,1.922614e+06,0.853746,0.850197


In [49]:



i=0

result = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'MSE_Train','R2_Train', 'Adj_r2_train' ,
                                'MSE_Test','R2_Test', 'Adj_r2_test' ])

while i < 5:

    # print('Simulation number :', i)


    # repeat this activity multiple times and record the R2 and MSE for each run
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)


    neigh = KNeighborsRegressor(n_neighbors=3)

    knn_reg = neigh.fit(X_train, y_train)
    R2_t= knn_reg.score(X, y)
    #print(reg.coef_)
    #print(reg.intercept_)
    y_pred_train = knn_reg.predict(X_train)
    # print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
    # print("Coefficient of determination: %.3f" % r2_score(y, y_pred))




    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)

    Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))


    y_pred_test = knn_reg.predict(X_test)
    # print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
    # print("Coefficient of determination: %.3f" % r2_score(y, y_pred))




    mse_test= mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))




    

    # result_fulldata = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

    case = 'knn k =3'

    # reg2 = LinearRegression().fit(X_train, y_train)
    # r2_t= reg2.score(X5_backward_4, y)
    # coefs = reg2.coef_
    # intercep = reg2.intercept_
    # # print(reg2.score(X5_backward_4, y))
    # # print(reg2.coef_)
    # # print(reg2.intercept_)

    # y_pred_train = reg2.predict(X_train)
    # mse_train = mean_squared_error(y_train, y_pred_train)
    # r2_train = r2_score(y_train, y_pred_train)
    # # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    # Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))






    # y_pred_test = reg2.predict(X_test)
    # mse_test = mean_squared_error(y_test, y_pred_test)
    # r2_test = r2_score(y_test, y_pred_test)
    # # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    # Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))





    i= i+1

    result=result.append({'R2 score': r2_t, 'MSE_Train': mse_train,
                             'R2_Train': r2_train, 'Adj_r2_train': Adj_r2_train, 
                             'MSE_Test': mse_test, 'R2_Test': r2_test, 
                             'Adj_r2_test': Adj_r2_test }, ignore_index=True)





r2_train_avg = result['R2_Train'].mean()
Adj_r2_train_avg= result['Adj_r2_train'].mean()
mse_train_avg = result['MSE_Train'].mean()



r2_test_avg = result['R2_Test'].mean()
Adj_r2_test_avg= result['Adj_r2_test'].mean()
mse_test_avg = result['MSE_Test'].mean()

# case= 'Serial feature selection backward 4 features'

# result_avg = pd.DataFrame( columns = ['Case','Average MSE_Train', 'Average R2_Train',
#                                         'Adj_r2_train_avg', 'Average MSE_Test', 'Average R2_Test',
#                                          'Adj_r2_test_avg'])

result_avg= result_avg.append({'Case': case, 'Average MSE_Train': mse_train_avg, 'Average R2_Train': r2_train_avg,
                                'Adj_r2_train_avg': Adj_r2_train_avg, 'Average MSE_Test': mse_test_avg, 
                                'Average R2_Test': r2_test_avg, 'Adj_r2_test_avg':Adj_r2_test_avg}, ignore_index=True)

result.head(12)

# print(' The total Mean of results after', i , ' number of simulations is :')

# result_avg.head()
    # df_f=df_f.append({'Name':name_4s,'Contact':c,'address':d,'url':u,'ratings':rat,'comments':com,'image':image}, ignore_index=True)





,R2 score,intercepts,coffeients,MSE_Train,R2_Train,Adj_r2_train,MSE_Test,R2_Test,Adj_r2_test
0,0.856789,NaN,NaN,1.913603e+06,0.846322,0.842593,5.208824e+06,0.647093,0.638529
1,0.856789,NaN,NaN,2.197362e+06,0.839371,0.835473,4.279048e+06,0.639393,0.630642
2,0.856789,NaN,NaN,2.162374e+06,0.835053,0.831050,5.054692e+06,0.617847,0.608573
3,0.856789,NaN,NaN,2.335709e+06,0.823148,0.818856,3.906278e+06,0.698267,0.690944
4,0.856789,NaN,NaN,2.144671e+06,0.832979,0.828926,5.011096e+06,0.637282,0.628479


In [50]:
print(' The total Mean of results after', i , ' number of simulations is :')

result_avg.head(15)

 The total Mean of results after 5  number of simulations is :


,Case,Average MSE_Train,Average R2_Train,Adj_r2_train_avg,Average MSE_Test,Average R2_Test,Adj_r2_test_avg
0,All features,1.201881e+06,0.911860,0.909721,2.269577e+06,0.820204,0.815840
1,Removed feature with P > 0.05,1.154337e+06,0.911701,0.910327,1.458381e+06,0.890374,0.888667
2,Serial feature selection forward 8 features,1.531963e+06,0.884140,0.883491,1.515289e+06,0.883279,0.882624
3,Serial feature selection backward 8 features,1.683699e+06,0.874541,0.873838,1.785790e+06,0.856672,0.855868
4,Serial feature selection Forward 4 features,1.814926e+06,0.860428,0.860038,1.830615e+06,0.862771,0.862388
5,Serial feature selection backward 4 features,1.867258e+06,0.858511,0.858115,1.922880e+06,0.852634,0.852222
6,knn k =3,2.150744e+06,0.835375,0.831379,4.691988e+06,0.647976,0.639433


In [51]:
from sklearn.neighbors import KNeighborsRegressor

p= len(X.columns)
print(p)
n= len(X.index)
print(n)


neigh = KNeighborsRegressor(n_neighbors=5)

knn_reg = neigh.fit(X, y)
R2_t_fulldata= knn_reg.score(X, y)
#print(reg.coef_)
#print(reg.intercept_)
y_pred = knn_reg.predict(X)
# print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
# print("Coefficient of determination: %.3f" % r2_score(y, y_pred))




mse = mean_squared_error(y, y_pred)
r2= r2_score(y, y_pred)

Adj_r2 = 1-(1-r2)*((n-1)/(n-p-1))

# result_fulldata = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

case_fulldata = 'knn k =5'

result_fulldata=result_fulldata.append({'case': case_fulldata, 'R2 score': R2_t_fulldata, 'Mean squared error': mse ,'R2' : r2, 'Adj_r2': Adj_r2}, ignore_index=True)


result_fulldata.head(15)


34
1436


,case,R2 score,intercepts,coffeients,Mean squared error,R2,Adj_r2
0,All features,0.908355,-2.197033e+13,"[1.6060105730085568, 913261203.5167035, 913261...",1.204742e+06,0.908355,0.906131
1,Removed feature with P > 0.05,0.907394,-4.496662e+04,"[1.7833675684437138, -157.6351853482074, -134....",1.217367e+06,0.907394,0.905953
2,Serial feature selection forward 8 features,0.884173,-4.882772e+06,"[3.284836191751551, 2439.9916913737075, 10.364...",1.522633e+06,0.884173,0.883523
3,Serial feature selection backward 8 features,0.870332,2.003402e+04,"[3.2546851851304592, -216.3452449689577, -176....",1.704578e+06,0.870332,0.869605
4,Serial feature selection forward 4 features,0.861914,-5.322925e+06,"[3.5162662254997987, 2665.805689981417, 639.45...",1.815233e+06,0.861914,0.861528
5,Serial feature selection backward 8 features,0.857154,2.174092e+04,"[3.384031470524448, -225.86612170543316, -181....",1.877815e+06,0.857154,0.856754
6,knn k =3,0.853746,NaN,NaN,1.922614e+06,0.853746,0.850197
7,knn k =5,0.789026,NaN,NaN,2.773407e+06,0.789026,0.783906


In [52]:



i=0

result = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'MSE_Train','R2_Train', 'Adj_r2_train' ,
                                'MSE_Test','R2_Test', 'Adj_r2_test' ])

while i < 5:

    # print('Simulation number :', i)


    # repeat this activity multiple times and record the R2 and MSE for each run
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)


    neigh = KNeighborsRegressor(n_neighbors=5)

    knn_reg = neigh.fit(X_train, y_train)
    R2_t= knn_reg.score(X, y)
    #print(reg.coef_)
    #print(reg.intercept_)
    y_pred_train = knn_reg.predict(X_train)
    # print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
    # print("Coefficient of determination: %.3f" % r2_score(y, y_pred))




    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)

    Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))


    y_pred_test = knn_reg.predict(X_test)
    # print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
    # print("Coefficient of determination: %.3f" % r2_score(y, y_pred))




    mse_test= mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))




    

    # result_fulldata = pd.DataFrame( columns = ['R2 score', 'intercepts', 'coffeients' ,'Mean squared error','R2', 'Adj_r2'])

    case = 'knn k =5'

    # reg2 = LinearRegression().fit(X_train, y_train)
    # r2_t= reg2.score(X5_backward_4, y)
    # coefs = reg2.coef_
    # intercep = reg2.intercept_
    # # print(reg2.score(X5_backward_4, y))
    # # print(reg2.coef_)
    # # print(reg2.intercept_)

    # y_pred_train = reg2.predict(X_train)
    # mse_train = mean_squared_error(y_train, y_pred_train)
    # r2_train = r2_score(y_train, y_pred_train)
    # # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    # Adj_r2_train = 1-(1-r2_train)*((n-1)/(n-p-1))






    # y_pred_test = reg2.predict(X_test)
    # mse_test = mean_squared_error(y_test, y_pred_test)
    # r2_test = r2_score(y_test, y_pred_test)
    # # print("MSE: %.2f" % mean_squared_error(y_test, y_pred_test))
    # # print("R2: %.3f" % r2_score(y_test, y_pred_test))
    # Adj_r2_test = 1-(1-r2_test)*((n-1)/(n-p-1))





    i= i+1

    result=result.append({'R2 score': r2_t, 'MSE_Train': mse_train,
                             'R2_Train': r2_train, 'Adj_r2_train': Adj_r2_train, 
                             'MSE_Test': mse_test, 'R2_Test': r2_test, 
                             'Adj_r2_test': Adj_r2_test }, ignore_index=True)





r2_train_avg = result['R2_Train'].mean()
Adj_r2_train_avg= result['Adj_r2_train'].mean()
mse_train_avg = result['MSE_Train'].mean()



r2_test_avg = result['R2_Test'].mean()
Adj_r2_test_avg= result['Adj_r2_test'].mean()
mse_test_avg = result['MSE_Test'].mean()

# case= 'Serial feature selection backward 4 features'

# result_avg = pd.DataFrame( columns = ['Case','Average MSE_Train', 'Average R2_Train',
#                                         'Adj_r2_train_avg', 'Average MSE_Test', 'Average R2_Test',
#                                          'Adj_r2_test_avg'])

result_avg= result_avg.append({'Case': case, 'Average MSE_Train': mse_train_avg, 'Average R2_Train': r2_train_avg,
                                'Adj_r2_train_avg': Adj_r2_train_avg, 'Average MSE_Test': mse_test_avg, 
                                'Average R2_Test': r2_test_avg, 'Adj_r2_test_avg':Adj_r2_test_avg}, ignore_index=True)

result.head(12)

# print(' The total Mean of results after', i , ' number of simulations is :')

# result_avg.head()
    # df_f=df_f.append({'Name':name_4s,'Contact':c,'address':d,'url':u,'ratings':rat,'comments':com,'image':image}, ignore_index=True)





,R2 score,intercepts,coffeients,MSE_Train,R2_Train,Adj_r2_train,MSE_Test,R2_Test,Adj_r2_test
0,0.856789,NaN,NaN,3.170586e+06,0.760261,0.754443,4.287992e+06,0.669070,0.661038
1,0.856789,NaN,NaN,3.337031e+06,0.744077,0.737867,4.252791e+06,0.682385,0.674677
2,0.856789,NaN,NaN,3.326033e+06,0.739742,0.733426,4.539230e+06,0.673741,0.665823
3,0.856789,NaN,NaN,2.768066e+06,0.784989,0.779771,5.750848e+06,0.580679,0.570503
4,0.856789,NaN,NaN,3.159213e+06,0.746438,0.740285,4.564371e+06,0.690424,0.682911


In [53]:
print(' The total Mean of results after', i , ' number of simulations is :')

result_avg.head(15)

 The total Mean of results after 5  number of simulations is :


,Case,Average MSE_Train,Average R2_Train,Adj_r2_train_avg,Average MSE_Test,Average R2_Test,Adj_r2_test_avg
0,All features,1.201881e+06,0.911860,0.909721,2.269577e+06,0.820204,0.815840
1,Removed feature with P > 0.05,1.154337e+06,0.911701,0.910327,1.458381e+06,0.890374,0.888667
2,Serial feature selection forward 8 features,1.531963e+06,0.884140,0.883491,1.515289e+06,0.883279,0.882624
3,Serial feature selection backward 8 features,1.683699e+06,0.874541,0.873838,1.785790e+06,0.856672,0.855868
4,Serial feature selection Forward 4 features,1.814926e+06,0.860428,0.860038,1.830615e+06,0.862771,0.862388
5,Serial feature selection backward 4 features,1.867258e+06,0.858511,0.858115,1.922880e+06,0.852634,0.852222
6,knn k =3,2.150744e+06,0.835375,0.831379,4.691988e+06,0.647976,0.639433
7,knn k =5,3.152186e+06,0.755102,0.749158,4.679046e+06,0.659260,0.650990


In [54]:
neigh = KNeighborsRegressor(n_neighbors=3)
#X = df[['TV','radio']]
#y = df[['sales']]
knn_reg = neigh.fit(X, y)
print(knn_reg.score(X, y))
#print(reg.coef_)
#print(reg.intercept_)
y_pred = knn_reg.predict(X)
print("Mean squared error: %.2f" % mean_squared_error(y, y_pred))
print("Coefficient of determination: %.3f" % r2_score(y, y_pred))

0.8537459241908385
Mean squared error: 1922613.86
Coefficient of determination: 0.854
